# Feature Selection on the 111101011 dataset

In this notebook we examine the result of dropping subsets of features that are linearly dependent or correlated to some degree.  We're specifically working on the dataset formed by dropping (31, 496, 524, 534, 917, 1183, 1299), since that resulted in the 4th lowest validation error.

In [30]:
import itertools
import numpy as np
import pandas as pd

pd.set_option('display.precision',20)
pd.set_option('display.max_colwidth',100)

from sklearn import linear_model
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, cross_val_predict, KFold, cross_val_score, \
                                    GridSearchCV, RandomizedSearchCV, ShuffleSplit 
from time import time
from scipy.stats import randint as sp_randint

import matplotlib.pylab as plt
from matplotlib.pylab import rcParams
from matplotlib import pyplot
rcParams['figure.figsize'] = 12, 4
%matplotlib inline

In [31]:
# def to compare goodness of fit on training set
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

In [32]:
# Cross-validation sets
kfold = KFold(n_splits=10, random_state=7)

# We are using LassoLarsCV as part of our metric
lr = linear_model.LassoLarsCV(verbose=False, max_iter=5000,precompute='auto', cv=kfold, max_n_alphas=1000, n_jobs=-1)

In [33]:
df = pd.read_csv("./input/train_tidy_111101011.csv")

In [34]:
ss = ShuffleSplit(n_splits=1, test_size=0.20, random_state=573)

In [35]:
X = df.values

In [36]:
for train_idx, validation_idx in ss.split(X):
    train_df = df.iloc[train_idx]
    validation_df = df.iloc[validation_idx]

We will establish a baseline by keeping all features.

In [37]:
y_validation = validation_df['SalePrice'].values
x_validation = validation_df.drop(['HouseId', 'SalePrice', 'GarageAge', 'GarageAgeLin'],axis=1).values
y_train = train_df['SalePrice'].values
x_train = train_df.drop(['HouseId', 'SalePrice', 'GarageAge', 'GarageAgeLin'],axis=1).values
lr.fit(x_train, y_train)
y_pred = lr.predict(x_validation)
baseline = rmse(y_validation, y_pred)
baseline

/usr/lib64/python2.7/site-packages/sklearn/linear_model/least_angle.py:309: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 316 iterations, i.e. alpha=4.700e-06, with an active set of 238 regressors, and the smallest cholesky pivot element being 1.490e-08
  ConvergenceWarning)
/usr/lib64/python2.7/site-packages/sklearn/linear_model/least_angle.py:334: ConvergenceWarning: Early stopping the lars path, as the residues are small and the current value of alpha is no longer well controlled. 349 iterations, alpha=5.193e-04, previous alpha=2.280e-06, with an active set of 254 regressors.
  ConvergenceWarning)
/usr/lib64/python2.7/site-packages/sklearn/linear_model/least_angle.py:334: ConvergenceWarning: Early stopping the lars path, as the residues are small and the current value of alpha is no longer well controlled. 346 iterations, alpha=3.966e-04, previous alpha=2.735e-06, with an active set of 251 regressors.
  ConvergenceWarning)
/usr/lib64/python2.7/

0.10515807312252047

## Features

We have a collection of Features, some of which were identified to be of potentially low-quality in predicting the response, others of which are known to be highly correlated with other Features.  We want to identify subsets of features that we can drop to improve the regression.

In [38]:
drop_cands = [
    'LotFrontage', 'LotArea', 'BsmtUnfSF', 'LowQualFinSF',
    'LogGrLivArea', 
    'GrLivArea', 'TotalHouseArea', 'LivArea', 'LivAreaWt', 'AllSizesSum', 'AllSizesSumLin', 'AreasSum',
    'X1stFlrSF','X1stLin', 'X2ndFlrSF', 'X2ndLin',
    'TotalBath', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
    'Age', 'AgeLin', 'RemodAgeLin','RemodAge',
    'MasVnrArea', 'MasVnrAreaLin',
    'DeckPorchLin','WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', 'X3SsnPorch', 'ScreenPorch'
]

In [39]:
corr_df = df[drop_cands].corr()

In [40]:
corr_df[corr_df > 0.75]

LotFrontage                 LotArea  BsmtUnfSF  LowQualFinSF  \
LotFrontage             1.0                     NaN        NaN           NaN   
LotArea                 NaN  1.00000000000000000000        NaN           NaN   
BsmtUnfSF               NaN                     NaN        1.0           NaN   
LowQualFinSF            NaN                     NaN        NaN           1.0   
LogGrLivArea            NaN                     NaN        NaN           NaN   
GrLivArea               NaN                     NaN        NaN           NaN   
TotalHouseArea          NaN                     NaN        NaN           NaN   
LivArea                 NaN                     NaN        NaN           NaN   
LivAreaWt               NaN                     NaN        NaN           NaN   
AllSizesSum             NaN  0.98606372397245245498        NaN           NaN   
AllSizesSumLin          NaN                     NaN        NaN           NaN   
AreasSum                NaN                     NaN        NaN           NaN   
X1stFlrSF               NaN                     NaN        NaN           NaN   
X1stLin                 NaN                     NaN        NaN           NaN   
X2ndFlrSF               NaN                     NaN        NaN           NaN   
X2ndLin                 NaN                     NaN        NaN           NaN   
TotalBath               NaN                     NaN        NaN           NaN   
BsmtFullBath            NaN                     NaN        NaN           NaN   
BsmtHalfBath            NaN                     NaN        NaN           NaN   
FullBath                NaN                     NaN        NaN           NaN   
HalfBath                NaN                     NaN        NaN           NaN   
Age                     NaN                     NaN        NaN           NaN   
AgeLin                  NaN                     NaN        NaN           NaN   
RemodAgeLin             NaN                     NaN        NaN           NaN   
RemodAge                NaN                     NaN        NaN           NaN   
MasVnrArea              NaN                     NaN        NaN           NaN   
MasVnrAreaLin           NaN                     NaN        NaN           NaN   
DeckPorchLin            NaN                     NaN        NaN           NaN   
WoodDeckSF              NaN                     NaN        NaN           NaN   
OpenPorchSF             NaN                     NaN        NaN           NaN   
EnclosedPorch           NaN                     NaN        NaN           NaN   
X3SsnPorch              NaN                     NaN        NaN           NaN   
ScreenPorch             NaN                     NaN        NaN           NaN   

                          LogGrLivArea               GrLivArea  \
LotFrontage                        NaN                     NaN   
LotArea                            NaN                     NaN   
BsmtUnfSF                          NaN                     NaN   
LowQualFinSF                       NaN                     NaN   
LogGrLivArea    1.00000000000000000000  0.97572734790673321115   
GrLivArea       0.97572734790673321115  1.00000000000000000000   
TotalHouseArea  0.86058937220902298471  0.83559555857857792383   
LivArea         0.76373332501539625827  0.76725195583060601390   
LivAreaWt       0.91883982161090238527  0.91377560660947887428   
AllSizesSum                        NaN                     NaN   
AllSizesSumLin                     NaN                     NaN   
AreasSum        0.83185505671546178252  0.81989666984062914423   
X1stFlrSF                          NaN                     NaN   
X1stLin                            NaN                     NaN   
X2ndFlrSF                          NaN                     NaN   
X2ndLin                            NaN                     NaN   
TotalBath                          NaN                     NaN   
BsmtFullBath                       NaN                     NaN   
BsmtHalfBath                       NaN   

We'll restrict our drop set to the highly correlated features to make this more readable,

In [41]:
drop_cands = [
    'LotArea', 'LogGrLivArea', 
    'GrLivArea', 'TotalHouseArea', 'LivArea', 'LivAreaWt', 'AllSizesSum', 'AreasSum',
    'X1stFlrSF','X1stLin', 'TotalBath', 'FullBath', 'MasVnrArea', 'MasVnrAreaLin'
]

In [42]:
corr_df = df[drop_cands].corr()

In [43]:
corr_df[corr_df > 0.75]

LotArea            LogGrLivArea  \
LotArea         1.00000000000000000000                     NaN   
LogGrLivArea                       NaN  1.00000000000000000000   
GrLivArea                          NaN  0.97572734790673321115   
TotalHouseArea                     NaN  0.86058937220902298471   
LivArea                            NaN  0.76373332501539625827   
LivAreaWt                          NaN  0.91883982161090238527   
AllSizesSum     0.98606372397245245498                     NaN   
AreasSum                           NaN  0.83185505671546178252   
X1stFlrSF                          NaN                     NaN   
X1stLin                            NaN                     NaN   
TotalBath                          NaN                     NaN   
FullBath                           NaN                     NaN   
MasVnrArea                         NaN                     NaN   
MasVnrAreaLin                      NaN                     NaN   

                             GrLivArea          TotalHouseArea  \
LotArea                            NaN                     NaN   
LogGrLivArea    0.97572734790673321115  0.86058937220902298471   
GrLivArea       1.00000000000000000000  0.83559555857857792383   
TotalHouseArea  0.83559555857857792383  1.00000000000000000000   
LivArea         0.76725195583060601390  0.80249169125840447148   
LivAreaWt       0.91377560660947887428  0.87595423074054235002   
AllSizesSum                        NaN                     NaN   
AreasSum        0.81989666984062914423  0.93233555658687905510   
X1stFlrSF                          NaN                     NaN   
X1stLin                            NaN                     NaN   
TotalBath                          NaN                     NaN   
FullBath                           NaN                     NaN   
MasVnrArea                         NaN                     NaN   
MasVnrAreaLin                      NaN                     NaN   

                               LivArea               LivAreaWt  \
LotArea                            NaN                     NaN   
LogGrLivArea    0.76373332501539625827  0.91883982161090238527   
GrLivArea       0.76725195583060601390  0.91377560660947887428   
TotalHouseArea  0.80249169125840447148  0.87595423074054235002   
LivArea         1.00000000000000000000  0.94575036201415385584   
LivAreaWt       0.94575036201415385584  1.00000000000000000000   
AllSizesSum                        NaN                     NaN   
AreasSum        0.79502614267426652273  0.86132578116270241964   
X1stFlrSF                          NaN                     NaN   
X1stLin                            NaN                     NaN   
TotalBath                          NaN                     NaN   
FullBath                           NaN                     NaN   
MasVnrArea                         NaN                     NaN   
MasVnrAreaLin                      NaN                     NaN   

                           AllSizesSum                AreasSum  \
LotArea         0.98606372397245245498                     NaN   
LogGrLivArea                       NaN  0.83185505671546178252   
GrLivArea                          NaN  0.81989666984062914423   
TotalHouseArea                     NaN  0.93233555658687905510   
LivArea                            NaN  0.79502614267426652273   
LivAreaWt                          NaN  0.86132578116270241964   
AllSizesSum     1.00000000000000000000                     NaN   
AreasSum                           NaN  1.00000000000000000000   
X1stFlrSF                          NaN                     NaN   
X1stLin                            NaN                     NaN   
TotalBath                          NaN                     NaN   
FullBath                           NaN                     NaN   
MasVnrArea                         NaN                     NaN   
MasVnrAreaLin                      NaN                     NaN   

                             X1stFlrSF                 X1stLin  \
L

Let's first compare the pairs.

In [44]:
drop_cands = ['LotArea', 'AllSizesSum']

In [45]:
col_drop_results_df = pd.DataFrame(dtype = 'float64')
count = 0
for L in range(0, len(drop_cands)+1):
        for subset in itertools.combinations(drop_cands, L):
                drop_cols = list(subset)
                col_drop_results_df.loc[count, 'Dropped'] = str(subset)
                x_train = train_df.drop(['HouseId', 'SalePrice', 'GarageAge', 'GarageAgeLin'] + drop_cols,axis=1).values
                x_validation = validation_df.drop(['HouseId', 'SalePrice', 'GarageAge', 'GarageAgeLin'] + drop_cols,axis=1).values
                lr.fit(x_train, y_train)
                y_pred = lr.predict(x_validation)
                error = rmse(y_validation, y_pred)
                col_drop_results_df.loc[count, 'RMSE'] = error
                col_drop_results_df.loc[count, 'Diff from Base'] = error - baseline                
                count += 1
output_df = col_drop_results_df.sort_values(['RMSE'])

/usr/lib64/python2.7/site-packages/sklearn/linear_model/least_angle.py:309: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 316 iterations, i.e. alpha=4.700e-06, with an active set of 238 regressors, and the smallest cholesky pivot element being 1.490e-08
  ConvergenceWarning)
/usr/lib64/python2.7/site-packages/sklearn/linear_model/least_angle.py:334: ConvergenceWarning: Early stopping the lars path, as the residues are small and the current value of alpha is no longer well controlled. 346 iterations, alpha=3.966e-04, previous alpha=2.735e-06, with an active set of 251 regressors.
  ConvergenceWarning)
/usr/lib64/python2.7/site-packages/sklearn/linear_model/least_angle.py:334: ConvergenceWarning: Early stopping the lars path, as the residues are small and the current value of alpha is no longer well controlled. 349 iterations, alpha=5.193e-04, previous alpha=2.280e-06, with an active set of 254 regressors.
  ConvergenceWarning)
/usr/lib64/python2.7/

In [46]:
output_df

Dropped                    RMSE          Diff from Base
3  ('LotArea', 'AllSizesSum')  0.10470987736431239257 -0.00044819575820807811
1                ('LotArea',)  0.10489855064689157516 -0.00025952247562889552
0                          ()  0.10515807312252047068  0.00000000000000000000
2            ('AllSizesSum',)  0.10558597126726469717  0.00042789814474422649

We should keep these features.

In [47]:
drop_cands = ['X1stFlrSF','X1stLin']

In [48]:
col_drop_results_df = pd.DataFrame(dtype = 'float64')
count = 0
for L in range(0, len(drop_cands)+1):
        for subset in itertools.combinations(drop_cands, L):
                drop_cols = list(subset)
                col_drop_results_df.loc[count, 'Dropped'] = str(subset)
                x_train = train_df.drop(['HouseId', 'SalePrice', 'GarageAge', 'GarageAgeLin'] + drop_cols,axis=1).values
                x_validation = validation_df.drop(['HouseId', 'SalePrice', 'GarageAge', 'GarageAgeLin'] + drop_cols,axis=1).values
                lr.fit(x_train, y_train)
                y_pred = lr.predict(x_validation)
                error = rmse(y_validation, y_pred)
                col_drop_results_df.loc[count, 'RMSE'] = error
                col_drop_results_df.loc[count, 'Diff from Base'] = error - baseline                
                count += 1
output_df = col_drop_results_df.sort_values(['RMSE'])

/usr/lib64/python2.7/site-packages/sklearn/linear_model/least_angle.py:334: ConvergenceWarning: Early stopping the lars path, as the residues are small and the current value of alpha is no longer well controlled. 345 iterations, alpha=1.262e-04, previous alpha=2.377e-06, with an active set of 256 regressors.
  ConvergenceWarning)
/usr/lib64/python2.7/site-packages/sklearn/linear_model/least_angle.py:334: ConvergenceWarning: Early stopping the lars path, as the residues are small and the current value of alpha is no longer well controlled. 349 iterations, alpha=5.193e-04, previous alpha=2.280e-06, with an active set of 254 regressors.
  ConvergenceWarning)
/usr/lib64/python2.7/site-packages/sklearn/linear_model/least_angle.py:334: ConvergenceWarning: Early stopping the lars path, as the residues are small and the current value of alpha is no longer well controlled. 347 iterations, alpha=2.184e-04, previous alpha=2.490e-06, with an active set of 258 regressors.
  ConvergenceWarning)
/usr

In [49]:
output_df

Dropped                    RMSE          Diff from Base
1            ('X1stFlrSF',)  0.10463252277742572660 -0.00052555034509474408
2              ('X1stLin',)  0.10473275521008570954 -0.00042531791243476114
3  ('X1stFlrSF', 'X1stLin')  0.10481817416540835375 -0.00033989895711211693
0                        ()  0.10515807312252047068  0.00000000000000000000

In [50]:
drop_cands = ['TotalBath', 'FullBath']

In [51]:
col_drop_results_df = pd.DataFrame(dtype = 'float64')
count = 0
for L in range(0, len(drop_cands)+1):
        for subset in itertools.combinations(drop_cands, L):
                drop_cols = list(subset)
                col_drop_results_df.loc[count, 'Dropped'] = str(subset)
                x_train = train_df.drop(['HouseId', 'SalePrice', 'GarageAge', 'GarageAgeLin'] + drop_cols,axis=1).values
                x_validation = validation_df.drop(['HouseId', 'SalePrice', 'GarageAge', 'GarageAgeLin'] + drop_cols,axis=1).values
                lr.fit(x_train, y_train)
                y_pred = lr.predict(x_validation)
                error = rmse(y_validation, y_pred)
                col_drop_results_df.loc[count, 'RMSE'] = error
                col_drop_results_df.loc[count, 'Diff from Base'] = error - baseline                
                count += 1
output_df = col_drop_results_df.sort_values(['RMSE'])

/usr/lib64/python2.7/site-packages/sklearn/linear_model/least_angle.py:309: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 316 iterations, i.e. alpha=4.700e-06, with an active set of 238 regressors, and the smallest cholesky pivot element being 1.490e-08
  ConvergenceWarning)
/usr/lib64/python2.7/site-packages/sklearn/linear_model/least_angle.py:334: ConvergenceWarning: Early stopping the lars path, as the residues are small and the current value of alpha is no longer well controlled. 349 iterations, alpha=5.193e-04, previous alpha=2.280e-06, with an active set of 254 regressors.
  ConvergenceWarning)
/usr/lib64/python2.7/site-packages/sklearn/linear_model/least_angle.py:334: ConvergenceWarning: Early stopping the lars path, as the residues are small and the current value of alpha is no longer well controlled. 347 iterations, alpha=2.184e-04, previous alpha=2.490e-06, with an active set of 258 regressors.
  ConvergenceWarning)
/usr/lib64/python2.7/

In [52]:
output_df

Dropped                    RMSE          Diff from Base
1             ('TotalBath',)  0.10439929425887752024 -0.00075877886364295044
3  ('TotalBath', 'FullBath')  0.10444268741494927566 -0.00071538570757119502
2              ('FullBath',)  0.10489855064689154740 -0.00025952247562892328
0                         ()  0.10515807312252047068  0.00000000000000000000

The gain by dropping both of these isn't big enough to consider dropping a predictor that we'd consider strong by hedonic reasoning.

In [53]:
drop_cands = ['MasVnrArea', 'MasVnrAreaLin']

In [54]:
col_drop_results_df = pd.DataFrame(dtype = 'float64')
count = 0
for L in range(0, len(drop_cands)+1):
        for subset in itertools.combinations(drop_cands, L):
                drop_cols = list(subset)
                col_drop_results_df.loc[count, 'Dropped'] = str(subset)
                x_train = train_df.drop(['HouseId', 'SalePrice', 'GarageAge', 'GarageAgeLin'] + drop_cols,axis=1).values
                x_validation = validation_df.drop(['HouseId', 'SalePrice', 'GarageAge', 'GarageAgeLin'] + drop_cols,axis=1).values
                lr.fit(x_train, y_train)
                y_pred = lr.predict(x_validation)
                error = rmse(y_validation, y_pred)
                col_drop_results_df.loc[count, 'RMSE'] = error
                col_drop_results_df.loc[count, 'Diff from Base'] = error - baseline                
                count += 1
output_df = col_drop_results_df.sort_values(['RMSE'])

/usr/lib64/python2.7/site-packages/sklearn/linear_model/least_angle.py:334: ConvergenceWarning: Early stopping the lars path, as the residues are small and the current value of alpha is no longer well controlled. 349 iterations, alpha=5.193e-04, previous alpha=2.280e-06, with an active set of 254 regressors.
  ConvergenceWarning)
/usr/lib64/python2.7/site-packages/sklearn/linear_model/least_angle.py:334: ConvergenceWarning: Early stopping the lars path, as the residues are small and the current value of alpha is no longer well controlled. 347 iterations, alpha=2.184e-04, previous alpha=2.490e-06, with an active set of 258 regressors.
  ConvergenceWarning)
/usr/lib64/python2.7/site-packages/sklearn/linear_model/least_angle.py:309: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 316 iterations, i.e. alpha=4.700e-06, with an active set of 238 regressors, and the smallest cholesky pivot element being 1.490e-08
  ConvergenceWarning)
/usr/lib64/python2.7/

In [55]:
output_df

Dropped                    RMSE  \
2               ('MasVnrAreaLin',)  0.10463252277742572660   
3  ('MasVnrArea', 'MasVnrAreaLin')  0.10468839374557338540   
1                  ('MasVnrArea',)  0.10468851377983115492   
0                               ()  0.10515807312252047068   

           Diff from Base  
2 -0.00052555034509474408  
3 -0.00046967937694708528  
1 -0.00046955934268931576  
0  0.00000000000000000000

In [56]:
drop_cands = ['LogGrLivArea', 'GrLivArea', 'TotalHouseArea', 'LivArea', 'LivAreaWt', 'AreasSum']

In [57]:
col_drop_results_df = pd.DataFrame(dtype = 'float64')
count = 0
for L in range(0, len(drop_cands)+1):
        for subset in itertools.combinations(drop_cands, L):
                drop_cols = list(subset)
                col_drop_results_df.loc[count, 'Dropped'] = str(subset)
                x_train = train_df.drop(['HouseId', 'SalePrice', 'GarageAge', 'GarageAgeLin'] + drop_cols,axis=1).values
                x_validation = validation_df.drop(['HouseId', 'SalePrice', 'GarageAge', 'GarageAgeLin'] + drop_cols,axis=1).values
                lr.fit(x_train, y_train)
                y_pred = lr.predict(x_validation)
                error = rmse(y_validation, y_pred)
                col_drop_results_df.loc[count, 'RMSE'] = error
                col_drop_results_df.loc[count, 'Diff from Base'] = error - baseline                
                count += 1
output_df = col_drop_results_df.sort_values(['RMSE'])

/usr/lib64/python2.7/site-packages/sklearn/linear_model/least_angle.py:334: ConvergenceWarning: Early stopping the lars path, as the residues are small and the current value of alpha is no longer well controlled. 347 iterations, alpha=2.184e-04, previous alpha=2.490e-06, with an active set of 258 regressors.
  ConvergenceWarning)
/usr/lib64/python2.7/site-packages/sklearn/linear_model/least_angle.py:309: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 316 iterations, i.e. alpha=4.700e-06, with an active set of 238 regressors, and the smallest cholesky pivot element being 1.490e-08
  ConvergenceWarning)
/usr/lib64/python2.7/site-packages/sklearn/linear_model/least_angle.py:334: ConvergenceWarning: Early stopping the lars path, as the residues are small and the current value of alpha is no longer well controlled. 346 iterations, alpha=3.966e-04, previous alpha=2.735e-06, with an active set of 251 regressors.
  ConvergenceWarning)
/usr/lib64/python2.7/

In [58]:
output_df

Dropped  \
59             ('LogGrLivArea', 'GrLivArea', 'TotalHouseArea', 'LivAreaWt', 'AreasSum')   
58               ('LogGrLivArea', 'GrLivArea', 'TotalHouseArea', 'LivArea', 'AreasSum')   
28                                       ('LogGrLivArea', 'TotalHouseArea', 'AreasSum')   
18                                                       ('TotalHouseArea', 'AreasSum')   
63  ('LogGrLivArea', 'GrLivArea', 'TotalHouseArea', 'LivArea', 'LivAreaWt', 'AreasSum')   
49                            ('LogGrLivArea', 'TotalHouseArea', 'LivArea', 'AreasSum')   
39                                            ('TotalHouseArea', 'LivArea', 'AreasSum')   
40                                          ('TotalHouseArea', 'LivAreaWt', 'AreasSum')   
47                               ('LogGrLivArea', 'GrLivArea', 'LivAreaWt', 'AreasSum')   
34                                          ('GrLivArea', 'TotalHouseArea', 'AreasSum')   
50                          ('LogGrLivArea', 'TotalHouseArea', 'LivAreaWt', 'AreasSum')   
53                               ('GrLivArea', 'TotalHouseArea', 'LivArea', 'AreasSum')   
21                                                            ('LivAreaWt', 'AreasSum')   
24                                           ('LogGrLivArea', 'GrLivArea', 'LivAreaWt')   
46                                 ('LogGrLivArea', 'GrLivArea', 'LivArea', 'AreasSum')   
61               ('LogGrLivArea', 'TotalHouseArea', 'LivArea', 'LivAreaWt', 'AreasSum')   
6                                                                         ('AreasSum',)   
11                                                         ('LogGrLivArea', 'AreasSum')   
20                                                              ('LivArea', 'AreasSum')   
30                                              ('LogGrLivArea', 'LivArea', 'AreasSum')   
36                                                 ('GrLivArea', 'LivArea', 'AreasSum')   
51                                 ('LogGrLivArea', 'LivArea', 'LivAreaWt', 'AreasSum')   
56                               ('TotalHouseArea', 'LivArea', 'LivAreaWt', 'AreasSum')   
15                                                            ('GrLivArea', 'AreasSum')   
60                    ('LogGrLivArea', 'GrLivArea', 'LivArea', 'LivAreaWt', 'AreasSum')   
31                                            ('LogGrLivArea', 'LivAreaWt', 'AreasSum')   
54                             ('GrLivArea', 'TotalHouseArea', 'LivAreaWt', 'AreasSum')   
4                                                                          ('LivArea',)   
17                                                      ('TotalHouseArea', 'LivAreaWt')   
7                                                         ('LogGrLivArea', 'GrLivArea')   
..                                                                                  ...   
26                                        ('LogGrLivArea', 'TotalHouseArea', 'LivArea')   
29                                             ('LogGrLivArea', 'LivArea', 'LivAreaWt')   
57              ('LogGrLivArea', 'GrLivArea', 'TotalHouseArea', 'LivArea', 'LivAreaWt')   
22                                      ('LogGrLivArea', 'GrLivArea', 'TotalHouseArea')   
23                                             ('LogGrLivArea', 'GrLivArea', 'LivArea')   
19                                                             ('LivArea', 'LivAreaWt')   
2                                                                        ('GrLivArea',)   
32                                           ('GrLivArea', 'TotalHouseArea', 'LivArea')   
42                           ('LogGrLivArea', 'GrLivArea', 'TotalHouseArea', 'LivArea')   
8                                                    ('LogGrLivArea', 'TotalHouseArea')   
1                                                                     ('LogGrLivArea',)   
9                                                           ('LogGrLivArea', 'LivArea')   
27                                      ('LogGrLivArea', 'TotalHouseArea', '